# CarMax Analytics Showcase: Winter 2023
## Rory Stevens
Desc

In [1]:
# Imports
import re
import time
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import seaborn as sns

### Data Loading and Preliminary Processing

In [20]:
raw_df = pd.read_csv("./Data/ShowcaseDataWinter2023.csv")
raw_memory = raw_df.memory_usage(deep=True).sum()
# To conserve energy, compute time --> limit_size = a number

print(f'Shape: {raw_df.shape}\nMemory Usage: {raw_memory} bytes ... {raw_memory/1000000:.2f} Mb')
raw_df.head()

Shape: (200000, 30)
Memory Usage: 224052594 bytes ... 224.05 Mb


,price,appraisal_offer,online_appraisal_flag,model_year,mileage,make,model,trim_descrip,body,color,...,trim_descrip_appraisal,body_appraisal,color_appraisal,engine_appraisal,cylinders_appraisal,mpg_city_appraisal,mpg_highway_appraisal,horsepower_appraisal,fuel_capacity_appraisal,market
0,$20k to $25k,$0k to $5k,1.0,2014.0,15k to 20k miles,HDT,HDT_12,Premium,E,Red,...,NaN,D,Silver,4.0L,6,17.0,21,236,21.0,13
1,$15k to $20k,$0k to $5k,0.0,2006.0,100k+ miles,KQZ,KQZ_23,Premium,E,Silver,...,Not Premium,D,Gold,4.2L,6,16.0,21,205,25.0,11
2,$30k to $35k,$35k to $40k,1.0,2014.0,5k to 10k miles,CZV,CZV_10,Premium,E,Black,...,NaN,E,Red,2.0L,4,41.0,38,181,10.0,13
3,$35k to $40k,$5k to $10k,1.0,2014.0,15k to 20k miles,IZA,IZA_19,Premium,F,Silver,...,Premium,F,Blue,3.0L,6,17.0,25,230,16.0,13
4,$30k to $35k,$15k to $20k,1.0,2011.0,35k to 40k miles,HDT,HDT_8,Premium,F,Blue,...,Not Premium,E,Black,2.4L,4,20.0,29,182,18.0,16


Taking a look at the 30 features, it is apparent that there are some features that need conversion to numeric values, namely the features denoted with ranges (ex. "20k to 25k miles").

In [21]:
# Can use a smaller portion of dataset for working on underpowered machines (or machines with small amount of available memory)
use_limited = False

if use_limited:
    raw_df = raw_df.iloc[:300,:]
    print(raw_df.shape)
    raw_df.head()

#### Take a look at dataset info with df.info()

In [22]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 30 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   price                    200000 non-null  object 
 1   appraisal_offer          200000 non-null  object 
 2   online_appraisal_flag    196166 non-null  float64
 3   model_year               200000 non-null  float64
 4   mileage                  200000 non-null  object 
 5   make                     200000 non-null  object 
 6   model                    200000 non-null  object 
 7   trim_descrip             176159 non-null  object 
 8   body                     189741 non-null  object 
 9   color                    199967 non-null  object 
 10  engine                   200000 non-null  object 
 11  cylinders                200000 non-null  int64  
 12  mpg_city                 199410 non-null  float64
 13  mpg_highway              199410 non-null  float64
 14  hors

We can see that the majority of features are entirely non-null, and there are a majority of features are objects.

#### Converting range features

The range representation of `price`, `appraisal_offer`, `mileage`, and `mileage_appraisal` is good for human readability, but for analyzing the data, it would be best to represent them as numerical values. So, if we take the midpoint of these ranges, we can preserve the relationship while changing the features to numeric values.

We don't know where the actual dollar amount or mileage number lies in these ranges, so no data is lost in this process.

In [23]:
def convert_range(range_ : str) -> np.float64:
    '''
    Converts rages (ex. price and mileage) to their midpoint as a float
    :param range_:
    :return:
    '''
    return np.average(list(map(int, re.findall(r'\d+', range_))))

In [24]:
print(raw_df.price.unique())
print('\n', raw_df.appraisal_offer.unique())
print('\n', raw_df.mileage.unique())
print('\n', raw_df.mileage_appraisal.unique())

['$20k to $25k' '$15k to $20k' '$30k to $35k' '$35k to $40k'
 '$45k to $50k' '$25k to $30k' '$40k to $45k' '$0 to $15k' '$55k to $60k'
 '$60k to $65k' '$65k to $70k' '$70k+' '$50k to $55k']

 ['$0k to $5k' '$35k to $40k' '$5k to $10k' '$15k to $20k' '$40k+'
 '$20k to $25k' '$10k to $15k' '$25k to $30k' '$30k to $35k']

 ['15k to 20k miles' '100k+ miles' '5k to 10k miles' '35k to 40k miles'
 '20k to 25k miles' '30k to 35k miles' '45k to 50k miles'
 '70k to 75k miles' '25k to 30k miles' '0 to 5k miles' '10k to 15k miles'
 '80k to 85k miles' '65k to 70k miles' '75k to 80k miles'
 '55k to 60k miles' '50k to 55k miles' '85k to 90k miles'
 '40k to 45k miles' '90k to 95k miles' '95k to 100k miles'
 '60k to 65k miles']

 ['200k+ miles' '0 to 10k miles' '10k to 20k miles' '100k to 110k miles'
 '20k to 30k miles' '160k to 170k miles' '30k to 40k miles'
 '80k to 90k miles' '60k to 70k miles' '110k to 120k miles'
 '70k to 80k miles' '90k to 100k miles' '190k to 200k miles'
 '150k to 160k miles' '1

Using `df.unique()`, we see that the maximum values for `price`, `appraisal_offer`, `mileage`, and `mileage_appraisal` are &#36;70k+, &#36;40k+, 100k+ miles, and 200k+ miles, respectively.

This means that the values have hard-capped ranges, which will be useful later in Memory Optimization.

In [25]:
# String features that represent numerical ranges
range_feats = ['price', 'appraisal_offer', 'mileage', 'mileage_appraisal']

# Convert each range-based feature value to the midpoint of the range. These are then kept as float as there exists an ordinal relationship.
for feature in range_feats:
    raw_df[feature] = raw_df[feature].apply(convert_range)
# Confirm changes with .dtypes and .head()

In [26]:
# Check conversion with df.head()
raw_df.head()

,price,appraisal_offer,online_appraisal_flag,model_year,mileage,make,model,trim_descrip,body,color,...,trim_descrip_appraisal,body_appraisal,color_appraisal,engine_appraisal,cylinders_appraisal,mpg_city_appraisal,mpg_highway_appraisal,horsepower_appraisal,fuel_capacity_appraisal,market
0,22.5,2.5,1.0,2014.0,17.5,HDT,HDT_12,Premium,E,Red,...,NaN,D,Silver,4.0L,6,17.0,21,236,21.0,13
1,17.5,2.5,0.0,2006.0,100.0,KQZ,KQZ_23,Premium,E,Silver,...,Not Premium,D,Gold,4.2L,6,16.0,21,205,25.0,11
2,32.5,37.5,1.0,2014.0,7.5,CZV,CZV_10,Premium,E,Black,...,NaN,E,Red,2.0L,4,41.0,38,181,10.0,13
3,37.5,7.5,1.0,2014.0,17.5,IZA,IZA_19,Premium,F,Silver,...,Premium,F,Blue,3.0L,6,17.0,25,230,16.0,13
4,32.5,17.5,1.0,2011.0,37.5,HDT,HDT_8,Premium,F,Blue,...,Not Premium,E,Black,2.4L,4,20.0,29,182,18.0,16


#### Changing engine sizes (contains 'L') to numerical values

We can safely remove the 'L' at the end of the values in the two engine features, because we know that all values in this column share the same unit, liters.

Important Note:
The units of engine size are almost always expressed as liters, but it is important to remember, as a general case, that it cannot always be assumed that units are consistent. You could lose information by removing unit designation.

In [27]:
slice_feats = ['engine', 'engine_appraisal']

for feature in slice_feats:
    # Get rid of last character
    raw_df[feature] = raw_df[feature].str.slice(start = 0, stop = -1)
    # Convert column to numeric and downcast
    raw_df[feature] = pd.to_numeric(raw_df[feature], downcast = "float")
print(f'New dtype for engine and engine_appraisal: {raw_df.engine.dtype}')

New dtype for engine and engine_appraisal: float32


## Memory Optimization

Memory optimization in a Jupyter notebook such as this would only be for the purposes of preventing memory errors and slow performance on constrained hardware. However, it is worth noting the data types and ways to optimize memory usage to get a better understanding of the data, their possible values, and optimal ways to store such data.

For nullable integers, we can use the nullable-integer extension dtypes in pandas
- Int8
- Int16
- Int32
- Int64


In [10]:
# Begin building the schema
# We already know the range features we converted are np.float64 given that we used np.average
# They also have hard limits, so we can safely use a smaller float dtype
schema={
    'price' : 'float16',
    'appraisal_offer': 'float16',
    'mileage' : 'float16',
    'mileage_appraisal' : 'float16'
}

#### Fuel Capacity

In [11]:
# Looking at fuel_capacity and fuel_capacity_appraisal..
print(raw_df['fuel_capacity'].unique())
print(raw_df['fuel_capacity_appraisal'].unique())
# All unique values can be represented by a nullable Int16 without losing data
schema['fuel_capacity'] = 'Int16'

[16. 18. 15. 19. 14. 31. 12. 13. 17. 24. 22. 38. 26. 28.  7. 21. 11. 23.
 20. 36.  8. 10. 35.  9. 27. 32. 48. 34. 25.  2. 29. nan 33.  1. 41. 30.
 37.]
[21. 25. 10. 16. 18. 19. 13. 22. 20. 14. 17. 26. 15. 23. 11. 12. 28. 24.
 31. nan 36.  9. 34. 35. 30.  8. 38. 27. 32. 33.  7. 42. 29. 48. 37. 39.]


It appears that fuel capacity values can be stored as a nullable Int16

#### Model Year
model_year and model_year_appraisal, with type float64, can be better stored as int32 since years are inherently unsigned 'small' integers (we will keep it as signed in case some special encoding is needed later). There are also no NaN's present, so a nullable type is not needed


In [12]:
schema['model_year'] = 'int32'
schema['model_year_appraisal'] = 'int32'

In [13]:
# Looking at the online appraisal feature, it appears that it is a binary feature with missing values.
print(raw_df.online_appraisal_flag.unique())
schema['online_appraisal_flag'] = 'Int8'
# Although this is converting the dtype to Int8, this feature will be re-encoded as a categorical feature later

[ 1.  0. nan]


#### Make
There is a set of distinct makes, so we can make this a categorical variable

In [14]:
print(raw_df.make.unique())
schema['make'] = 'category'

['HDT' 'KQZ' 'CZV' 'IZA' 'OFW' 'LTX' 'FVQ' 'ARU' 'HXQ' 'BDP' 'CFK' 'CIP'
 'AIH' 'JHO' 'BGZ' 'FJE' 'MBN' 'JJQ' 'NHY' 'NKR' 'GUK' 'LYP' 'JDZ' 'BZQ'
 'HAE' 'BXO' 'FPH' 'HYS' 'JGV' 'HAN' 'NAJ' 'NYP' 'IZN' 'DVE' 'KDM' 'FJR'
 'BIS']


#### Model
The model distinction is far more specific than make (intuitively), so it makes more sense to keep this as an object. Additionally, if we extracted the model number from this feature and simply made that a column of itself for better readability/differentiation, it would misrepresent the data because a model 10 on one make is not the same as a model 10 on another.

In [15]:
print(f'Number of models : {len(raw_df.model.unique())}')

Number of models : 575


#### Trim Description, Body, Color
These features appear to be categorical in nature with some NaN values, so the best dtype would be category.

In [16]:
print(f'trim_descrip: {raw_df.trim_descrip.unique()}')
print(f'\nbody: {raw_df.body.unique()}')
print(f'\ncolor: {raw_df.color.unique()}')
schema['trim_descrip'] = 'category'
schema['trim_descrip_appraisal'] = 'category'
schema['body'] = 'category'
schema['body_appraisal'] = 'category'
schema['color'] = 'category'
schema['color_appraisal'] = 'category'

trim_descrip: ['Premium' 'Not Premium' nan]

body: ['E' 'F' 'B' nan 'D' 'G' 'A']

color: ['Red' 'Silver' 'Black' 'Blue' 'White' 'Gray' 'Burgundy' 'Gold' 'Brown'
 'Green' 'Pearl' 'Purple' 'Tan' 'Orange' nan 'Yellow' 'Unknown']


#### Engine
We already found that the two engine features were best changed to floats because the unit designation of 'L' was constant and they are continuous (somewhat) values.

Additionally, since engine sizes cannot be extremely large values, we already downcast the output from our conversion to numeric.

In [17]:
print(f'Data type of engine and engine_appraisal after conversion: {raw_df.engine.dtype}')

Data type of engine and engine_appraisal after conversion: float32


#### Cylinders
The cylinders feature has an ordinal relationship and a typically defined range, so we can change this to int16 since it also has no null values.

In [18]:
print(f'Unique cylinder values: {raw_df.cylinders.unique()}')
print(f'Unique appraised cylinder values: {raw_df.cylinders_appraisal.unique()}')
schema['cylinders'] = 'int16'
schema['cylinders_appraisal'] = 'int16'

Unique cylinder values: [ 4  6  8  3  0  5  2 10 12 16]
Unique appraised cylinder values: [ 6  4  8  5  3 10]


#### MPG, Fuel Capacity, and  Horsepower
mpg_city, mpg_highway, fuel_capacity, and horsepower for the purchased and appraised vehicle have dtypes of float64, float64, float64, and int64. Because mpg_city, mpg_highway have null values, we can keep these as float64 (a nullable type). For horsepower, an integer column which has no null values, we can save memory and downcast this column.

In [37]:
print(f'Horsepower max: {raw_df.horsepower.max()}\nHorsepower Min: {raw_df.horsepower.min()}')
schema['horsepower'] = 'int32'
schema['horsepower_appraisal'] = 'int32'

Horsepower max: 760
Horsepower Min: 11


#### Market
Market is listed as a string value representing the market in which the transaction occurred. This means that this number has no ordinal relationship, so it should be made a categorical column.

In [41]:
print(f'Market Max: {raw_df.market.max()}\nMarket Min: {raw_df.market.min()}')
print(sorted(raw_df.market.unique()))
schema['market'] = 'category'

Market Max: 16
Market Min: 1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


#### Data type inspection and Memory Optimization: Conclusions

Using the schema created during our inspection of the feature's data types, we can get a 76% reduction in memory usage! Again, this is not a requirement for performing data science tasks, but it is certainly a good exercise on learning where performance can be gained when working with large amounts of data. In a data storage layer, this data is certainly optimized to be as small as possible to save on costs; however, its

In [48]:
# Astype conversion
trimmed_raw_df = raw_df.astype(schema)
trimmed_raw_memory = trimmed_raw_df.memory_usage(deep=True).sum()

print(f'Raw Dataframe Memory Usage: {raw_memory} bytes ... {raw_memory/1000000:.2f} Mb')
print(f'Optimized Dataframe Memory Usage: {trimmed_raw_memory} bytes ... {trimmed_raw_memory/1000000:.2f} Mb')
print(f'% Memory Reduction from raw dataframe to optimized dataframe {((raw_memory - trimmed_raw_memory) / raw_memory * 100) : .2f} %')
trimmed_raw_df.info()

Raw Dataframe Memory Usage: 224052594 bytes ... 224.05 Mb
Optimized Dataframe Memory Usage: 53365615 bytes ... 53.37 Mb
% Memory Reduction from raw dataframe to optimized dataframe  76.18 %
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 30 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   price                    200000 non-null  float16 
 1   appraisal_offer          200000 non-null  float16 
 2   online_appraisal_flag    196166 non-null  Int8    
 3   model_year               200000 non-null  uint16  
 4   mileage                  200000 non-null  float16 
 5   make                     200000 non-null  category
 6   model                    200000 non-null  object  
 7   trim_descrip             176159 non-null  category
 8   body                     189741 non-null  category
 9   color                    199967 non-null  category
 10  engine                

In [49]:
np.info('float16')

     *** Found in numpy ***
 float16()

Half-precision floating-point number type.

:Character code: ``'e'``
:Canonical name: `numpy.half`
:Alias on this platform (Windows AMD64): `numpy.float16`: 16-bit-precision floating-point number type: sign bit, 5 bits exponent, 10 bits mantissa.


Methods:

  all  --  Scalar method identical to the corresponding array attribute.
  any  --  Scalar method identical to the corresponding array attribute.
  argmax  --  Scalar method identical to the corresponding array attribute.
  argmin  --  Scalar method identical to the corresponding array attribute.
  argsort  --  Scalar method identical to the corresponding array attribute.
  as_integer_ratio  --  half.as_integer_ratio() -> (int, int)
  astype  --  Scalar method identical to the corresponding array attribute.
  byteswap  --  Scalar method identical to the corresponding array attribute.
  choose  --  Scalar method identical to the corresponding array attribute.
  clip  --  Scalar method identica